In [14]:
pip install --upgrade geopandas

Traceback (most recent call last):
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/site-packages/pip/__main__.py", line 8, in <module>
    if sys.path[0] in ("", os.getcwd()):
FileNotFoundError: [Errno 2] No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install pandas==1.3.5

Traceback (most recent call last):
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/site-packages/pip/__main__.py", line 8, in <module>
    if sys.path[0] in ("", os.getcwd()):
FileNotFoundError: [Errno 2] No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install rasterio

Traceback (most recent call last):
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/gominji/anaconda3/envs/airlab1/lib/python3.10/site-packages/pip/__main__.py", line 8, in <module>
    if sys.path[0] in ("", os.getcwd()):
FileNotFoundError: [Errno 2] No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.transform import rowcol

root_directory = './seoul,gyeong-gi/2023'

jp2_files = []

for root, dirs, files in os.walk(root_directory):
    for dir in dirs:
        granule_path = os.path.join(root, dir, 'GRANULE')
        if os.path.exists(granule_path):
            for granule_subdir in os.listdir(granule_path):
                img_data_path = os.path.join(granule_path, granule_subdir, 'IMG_DATA', 'R20m')
                if os.path.exists(img_data_path):
                    files = glob.glob(os.path.join(img_data_path, '*.jp2'))
                    jp2_files.extend(files)

csv_file_path = './Address.csv'

df = pd.read_csv(csv_file_path)

# Assuming your CSV has columns named 'Longitude' and 'Latitude'
# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
    crs="EPSG:4326"  # Assuming the coordinates are in WGS84
)

# Function to extract values from images
def extract_values_from_image(image_path, gdf, column_name):
    with rasterio.open(image_path) as dataset:
        # Ensure the GeoDataFrame has the same CRS as the image
        if gdf.crs != dataset.crs:
            gdf = gdf.to_crs(dataset.crs)

        # Loop through each point in the GeoDataFrame
        for idx, row in gdf.iterrows():
            # Extract the point coordinates
            point = row.geometry
            x, y = point.x, point.y

            # Get the row and column of the transformed coordinate
            row_idx, col_idx = rowcol(dataset.transform, x, y)

            # Read the value at the row and column
            value = dataset.read(1)[row_idx, col_idx]

            # Add the value to the GeoDataFrame
            gdf.loc[idx, column_name] = value

    return gdf

# Extract values from each image and add them to the GeoDataFrame
for jp2_file in jp2_files:
    band_name = os.path.basename(jp2_file).split('_')[1]  # Example to derive a band name
    gdf = extract_values_from_image(jp2_file, gdf, band_name)

# Save the GeoDataFrame to a CSV file
output_csv_path = './korea_bandvalues/seoul,gyeong-gi_band_value.csv'
gdf.to_csv(output_csv_path, index=False)

# Print the absolute path to confirm where the file was saved
absolute_path = os.path.abspath(output_csv_path)
print(f"CSV file saved at: {absolute_path}")


FileNotFoundError: [Errno 2] No such file or directory: './Address.csv'